In [1]:
from env2048.vectenv import Vectorized2048Env
from env2048.vectmcts import Vectorized2048MCTSLazy
from az_resnet import AZResnet, AZResnetArchitectureParameters
import torch

/Users/marshingjay/miniconda3/envs/python3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_boards = 100
v = Vectorized2048Env(num_boards, device)
model = AZResnet(AZResnetArchitectureParameters(
    input_size=torch.Size((1, 4, 4)),
    policy_size=4,
    res_channels=8,
    res_blocks=2,
    value_head_res_channels=8,
    value_head_res_blocks=2,
    policy_head_res_channels=8,
    policy_head_res_blocks=2,
    kernel_size=3,
    policy_fc_size=32,
    value_fc_size=32
))
mc = Vectorized2048MCTSLazy(v, model, 1)
v.reset()



In [6]:
def run_for_steps(steps, iters, depth):
    for _ in range(steps):
        moves = mc.choose_action(iters, depth)
        v.step(moves)

In [7]:
%prun -s tottime run_for_steps(5, 5, 5)

         99819 function calls (89029 primitive calls) in 8.249 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1950    5.371    0.003    5.371    0.003 {built-in method torch.batch_norm}
      155    0.810    0.005    1.066    0.007 vectenv.py:75(get_progressions)
     2330    0.769    0.000    0.769    0.000 {built-in method torch.conv2d}
      520    0.389    0.001    0.389    0.001 {built-in method torch._C._nn.linear}
     1115    0.262    0.000    0.262    0.000 {built-in method torch.any}
     1950    0.055    0.000    0.055    0.000 {built-in method torch.relu}
      155    0.052    0.000    0.082    0.001 vectenv.py:103(merge)
      310    0.045    0.000    0.060    0.000 vectenv.py:94(rotate_by_amnts)
    18980    0.043    0.000    0.043    0.000 module.py:1194(__getattr__)
 9360/130    0.042    0.000    6.787    0.052 module.py:1124(_call_impl)
     1950    0.039    0.000    5.488    0.003 batchnorm.py:134(fo